# Global Settings and Imports

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import os
import sys

sys.path.insert(0, '..')

import seq_rec.utils.custom_logging

In [ ]:
# from seq_rec.experiment_tracker import CometExperiment as Experiment
from seq_rec.experiment_tracker import MockExperiment as Experiment
experiment = Experiment(
    auto_metric_logging=True,
    auto_param_logging=False,
    auto_histogram_weight_logging=True,
    auto_histogram_gradient_logging=True,
    auto_histogram_activation_logging=True,
    auto_histogram_tensorboard_logging=True
)

In [ ]:
import seq_rec.io as io
import seq_rec.utils as utils
from seq_rec.models.deeprec.tfrs_gru import TFRSGRUModel as Model

## Parameters

In [ ]:
RANDOM_SEED = 13  # Set None for non-deterministic result

HYDRA_CONFIG_PATH = '../seq_rec/conf/'
COUNTRY_CODE = 'SG'

# Config object

In [ ]:
cfg = utils.load_cfg(
    HYDRA_CONFIG_PATH,
    overrides=[
        f"env.io.training.applied_country_code=['{COUNTRY_CODE}']"
    ]
)

## Load the input data

In [ ]:
training_ds = io.load_training_data(cfg.env.io.training, part='train', random_seed=RANDOM_SEED, download=True)
val_ds = io.load_training_data(cfg.env.io.training, part='validation', random_seed=RANDOM_SEED)
test_ds = io.load_training_data(cfg.env.io.training, part='test', random_seed=RANDOM_SEED)

# Prep the input data

In [ ]:
training_prep_ds = io.prep_training_data(training_ds, cfg.input_prep)
val_prep_ds = io.prep_training_data(val_ds, cfg.input_prep)
test_prep_ds = io.prep_training_data(test_ds, cfg.input_prep)

In [ ]:
merchant_vocab = io.get_vocab(training_ds, 'merchant_id')
search_vocab = io.get_vocab(training_prep_ds, 'context_search_terms')

# Prepare hyper-parameters

In [ ]:
model_hparams_folder = '../seq_rec/models/deeprec/conf'
model_codename = 'tfrs_gru'
model_version = "v01"

In [ ]:
model_hparams = {
    **utils.load_cfg(
        model_hparams_folder,
        overrides=[
            f"tfrs_gru={model_version}",
            # "tfrs_gru.epochs=1"
        ]
    )[model_codename],
    "merchant_vocab": merchant_vocab,
    "search_vocab": search_vocab,
    "random_seed": RANDOM_SEED,
    "country_code": COUNTRY_CODE
}

In [ ]:
print(model_hparams)

# Create model

In [ ]:
model = Model(model_hparams)

#### Eval before training

In [ ]:
print(model.run_eval(val_prep_ds))

## Train model

In [ ]:
with experiment.train():
    model.fit(training_prep_ds, val_prep_ds)

## Evaluate model

In [ ]:
with experiment.test():
    metrics_test = model.run_eval(test_prep_ds)
    experiment.log_metrics(metrics_test)

## Clean up experiment

In [ ]:
experiment.log_parameters(model_hparams)

In [ ]:
experiment.end()

# Export model

In [ ]:
MODEL_NAME = f'seq_rec-{COUNTRY_CODE}-v0'
SAVED_MODEL_PATH = f'models/{MODEL_NAME}'
model.export(SAVED_MODEL_PATH)

SAVED_EMBEDDINGS_BASE_LOG_DIR = f'logs/{MODEL_NAME}/embeddings/'
model.export_embeddings(base_log_dir=SAVED_EMBEDDINGS_BASE_LOG_DIR)

## Test loaded model

In [ ]:
import tensorflow as tf
import numpy as np

instance = {
    'context_merchants': np.array([[b'<EXAMPLE_MERCHANT_ID>']]),
    'context_search_terms': np.array([[b'NULL']]),
    'recent_txn_merchants': np.array([[b'NULL']]),
    "context_merchants_time_recency": np.array([[b'1']]),
    "context_search_terms_time_recency": np.array([[b'0']]),
    "recent_txn_time_recency": np.array([[b'0']]),
}

loaded = tf.saved_model.load(SAVED_MODEL_PATH)
serving_fn = loaded.signatures['serving_default']
recommendations = serving_fn(**instance)
print(f"Recommendations: {recommendations['merchant_id'][0]}")
